In [36]:
# Import Dependencies
import os
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import json
import pymongo

In [37]:
with open("../Resources/chargerStations.js") as f:
    charger_json = json.load(f)
with open("../Resources/gasStations.js") as f:
    gas_json = json.load(f)
display(charger_json.keys())
display(gas_json.keys())

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.FullData

dict_keys(['type', 'name', 'crs', 'features'])

dict_keys(['type', 'name', 'crs', 'features'])

In [38]:
#Gas Stations data paired down and listified
gas_names = []
gas_longs = []
gas_lats = []
for x in range(0,len(gas_json["features"])):
    gas_names.append(gas_json["features"][x]["properties"]["name"])
    gas_longs.append(gas_json["features"][x]["geometry"]["coordinates"][0])
    gas_lats.append(gas_json["features"][x]["geometry"]["coordinates"][1])
    
# Charger Stations data paired down and listified  
charger_names = []
charger_longs = []
charger_lats = []
for x in range(0, len(charger_json["features"])):
    charger_names.append(charger_json["features"][x]["properties"]["name"])
    charger_longs.append(charger_json["features"][x]["geometry"]["coordinates"][0])
    charger_lats.append(charger_json["features"][x]["geometry"]["coordinates"][1])

In [39]:
# Create DataFrames
charger_df = pd.DataFrame({
    "Charger Station Name" : charger_names,
    "Charger Station Longitude" : charger_longs,
    "Charger Station Latitude" : charger_lats
})
gas_df = pd.DataFrame({
    "Gas Station Name" : gas_names,
    "Gas Station Longitude" : gas_longs,
    "Gas Station Latitude" : gas_lats
})

In [40]:
#Bring in campsites and mountains csv
campsites = pd.read_csv("../Resources/southwest_camp_data.csv", encoding = "utf-8")
campsites = campsites[["Long", "Lat", "name", "type", "dates", "# sites", "Elevation", "State"]]
co_campsites = campsites[campsites["State"] == "CO"]
display(co_campsites.head())

mountains = pd.read_csv("../Resources/14ers_data.csv", encoding = "ISO-8859–1")
display(mountains.head())

,Long,Lat,name,type,dates,# sites,Elevation,State
259,-106.345,37.379,Alamosa,NF,,10,8665,CO
260,-106.858,38.654,Almont,NF,mid may-mid sep,10,8018,CO
261,-105.562,38.080,Alvarado,NF,early jun-mid oct,50,8986,CO
262,-107.665,38.019,Amphitheater,NF,mid jun-late sep,35,8320,CO
263,-106.221,38.583,Angel of Shavano,NF,,20,9180,CO


,ID,Mountain Peak,Mountain Range,Elevation_ft,fourteener,Prominence_ft,Isolation_mi,Peak Longitude,Peak Latitude,Standard Route,Standard Route Distance (miles),Elevation Gain_ft,Difficulty,Traffic Low,Traffic High,photo
0,1,Mt. Elbert,Sawatch Range,14440,Y,9093,670.00,-106.4454,39.1178,Northeast Ridge,9.50,4700,Class 1,20000,25000,https://www.14ers.com/photos/mtelbert/peakphot...
1,2,Mt. Massive,Sawatch Range,14428,Y,1961,5.06,-106.4757,39.1875,East Slopes,14.50,4500,Class 2,7000,10000,https://www.14ers.com/photos/mtmassive/peakpho...
2,3,Mt. Harvard,Sawatch Range,14421,Y,2360,14.93,-106.3207,38.9244,South Slopes,14.00,4600,Class 2,5000,7000,https://www.14ers.com/photos/harvardgroup/peak...
3,4,Blanca Peak,Sangre de Cristo Range,14351,Y,5326,103.40,-105.4856,37.5775,Northwest Ridge,17.00,6500,Hard Class 2,1000,3000,https://www.14ers.com/photos/blancagroup/peakp...
4,5,La Plata Peak,Sawatch Range,14343,Y,1836,6.28,-106.4729,39.0294,Northwest Ridge,9.25,4500,Class 2,5000,7000,https://www.14ers.com/photos/laplatapeak/peakp...


In [41]:
# Create empty list to append information
distance_list = []
mountain_list = []
camp_list = []
co_campsites_lats = []
co_campsites_longs = []

#iterate through mountains and co_example append results to empty list. use math to find radius of 20 miles
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(co_campsites["Lat"],co_campsites["Long"], co_campsites["name"], ):
        lat2 = radians(i)
        lon2 = radians(j)
        camp_name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_list.append(l)
            distance_list.append(distance)
            camp_list.append(camp_name)
            co_campsites_lats.append(i)
            co_campsites_longs.append(j)

In [42]:
# Create Pandas DF and utilizing the lists above. Drop any duplicates from the new DF
distance_df = pd.DataFrame({
    "Mountain Peak": mountain_list,
    "Distance from Campsite (miles)" : distance_list,
    "name": camp_list,
    "Campsite Latitude" : co_campsites_lats,
    "Campsite Longitude" : co_campsites_longs})
distance_df = distance_df.drop_duplicates()
display(distance_df.head())

everything_14ers = db.everything_14ers_db
data = distance_df.to_dict(orient = "records")
everything_14ers.insert_many(data)

,Mountain Peak,Distance from Campsite (miles),name,Campsite Latitude,Campsite Longitude
0,Mt. Elbert,11.667981,Baby Doe,39.270,-106.351
1,Mt. Elbert,11.652809,Belle of Colorado,39.269,-106.349
2,Mt. Elbert,16.725207,Chapman,39.309,-106.637
3,Mt. Elbert,6.697035,Dexter,39.092,-106.325
4,Mt. Elbert,17.685043,Difficult,39.141,-106.774


In [43]:
camp_merge = pd.merge(distance_df, co_campsites, on = "name")
mountains_merge = pd.merge(camp_merge, mountains, on = "Mountain Peak")
cleaned_mountains = mountains_merge[["Mountain Peak", "Distance from Campsite (miles)", "name","Long", "Lat", "dates", "# sites", "Isolation_mi", "Peak Longitude", "Peak Latitude"]]
cleaned_mountains = cleaned_mountains.rename(columns = {"name" : "Campsite Name", "Long": "Campsite Longitude", "Lat": "Campsite Latitude", "dates" : "Dates", "# sites" : "#_Sites"})
display(cleaned_mountains.head())

,Mountain Peak,Distance from Campsite (miles),Campsite Name,Campsite Longitude,Campsite Latitude,Dates,#_Sites,Isolation_mi,Peak Longitude,Peak Latitude
0,Mt. Elbert,11.667981,Baby Doe,-106.351,39.270,early jun-late aug,50,670.0,-106.4454,39.1178
1,Mt. Elbert,11.652809,Belle of Colorado,-106.349,39.269,,19,670.0,-106.4454,39.1178
2,Mt. Elbert,16.725207,Chapman,-106.637,39.309,mid may-mid sep,84,670.0,-106.4454,39.1178
3,Mt. Elbert,6.697035,Dexter,-106.325,39.092,,24,670.0,-106.4454,39.1178
4,Mt. Elbert,17.685043,Difficult,-106.774,39.141,mid may-mid sep,47,670.0,-106.4454,39.1178


Find Distances from Various Points

In [44]:
charger_distance_list = []
mountain_charger_list = []
charger_lats1 = []
charger_longs1 = []
charger_names1 = []
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(charger_df["Charger Station Latitude"], charger_df["Charger Station Longitude"], charger_df["Charger Station Name"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_charger_list.append(l)
            charger_distance_list.append(distance)
            charger_lats1.append(i)
            charger_longs1.append(j)
            charger_names1.append(Name)

In [45]:
charger_mountain_df = pd.DataFrame({
    "Charger Station Name": charger_names1,
    "Mountain Peak": mountain_charger_list,
    "Charging Distance from Peak" : charger_distance_list,
    "Charging Station Latitude": charger_lats1,
    "Charging Station Longitude" : charger_longs1,
})
display(charger_mountain_df.head())

everything_14ers = db.everything_14ers_db
data = charger_mountain_df.to_dict(orient = "records")
everything_14ers.insert_many(data)

,Charger Station Name,Mountain Peak,Charging Distance from Peak,Charging Station Latitude,Charging Station Longitude
0,None,Mt. Massive,18.320254,39.191468,-106.817763
1,BV Charge,Mt. Harvard,11.705837,38.841832,-106.130655
2,South Main Public Parking,Mt. Harvard,12.207821,38.841441,-106.120300
3,Riverfront Park,Mt. Harvard,11.930070,38.846897,-106.122483
4,Tesla Supercharger,Mt. Lincoln,19.462486,39.631412,-106.070762


In [46]:
gas_distance_list = []
mountain_gas_list = []
gas_lats1 = []
gas_longs1 = []
gas_names1 = []
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(gas_df["Gas Station Latitude"], gas_df["Gas Station Longitude"], gas_df["Gas Station Name"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_gas_list.append(l)
            gas_distance_list.append(distance)
            gas_lats1.append(i)
            gas_longs1.append(j)
            gas_names1.append(Name)

In [47]:
gas_mountain_df = pd.DataFrame({
    "Gas Station Name": gas_names1,
    "Mountain Peak": mountain_gas_list,
    "Gas Station Distance from Peak" : gas_distance_list,
    "Gas Station Latitude": gas_lats1,
    "Gas Station Longitude" : gas_longs1,
})
display(gas_mountain_df.head())

everything_14ers = db.everything_14ers_db
data = gas_mountain_df.to_dict(orient = "records")
everything_14ers.insert_many(data)

,Gas Station Name,Mountain Peak,Gas Station Distance from Peak,Gas Station Latitude,Gas Station Longitude
0,Shell,Mt. Elbert,8.980248,39.223783,-106.348358
1,Diamond Shamrock,Mt. Elbert,12.090677,39.244833,-106.290142
2,Kum & Go,Mt. Elbert,12.145856,39.246371,-106.290747
3,Phillips 66,Mt. Elbert,12.834498,39.260375,-106.291778
4,Conoco,Mt. Elbert,12.850290,39.260834,-106.292028


In [48]:
breweries_df = pd.read_csv("../Resources/cleaned_breweries.csv", encoding = "utf-8")
display(breweries_df.head())

,Brewery,Address,Latitudes,Longitudes
0,Twisted Pine Brewing Company,"3201 Walnut Street, Boulder, CO 80301",40.020826,-105.251023
1,Aspen Brewing Company,"304 E Hopkins Ave, Aspen, CO 81611",39.190323,-106.820301
2,Copper Kettle Brewing Co,"1338 South Valentia Street, Unit 100, Denver, ...",39.692358,-104.890912
3,Yak & Yeti - Arvada,"7803 Ralston Rd., Arvada, CO 80002",39.802384,-105.084192
4,Outer Range Brewing Company,"182 Lusher Ct,, Frisco, CO 80443",39.589334,-106.099061


In [49]:
beer_distance_list = []
mountain_beer_list = []
beer_lats = []
beer_longs = []
beer_names = []
beer_addresses = []
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k, o in zip(breweries_df["Latitudes"], breweries_df["Longitudes"], breweries_df["Brewery"], breweries_df["Address"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_beer_list.append(l)
            beer_distance_list.append(distance)
            beer_lats.append(i)
            beer_longs.append(j)
            beer_names.append(Name)
            beer_addresses.append(o)

In [50]:
brewery_mountain_df = pd.DataFrame({
    "Brewery Name" : beer_names,
    "Brewery Address" : beer_addresses,
    "Mountain Peak" : mountain_beer_list,
    "Brewery Distance From Mountain Peak" : beer_distance_list,
    "Brewery Longitudes" : beer_longs,
    "Brewery Latitudes" : beer_lats,
})
display(brewery_mountain_df.head())

everything_14ers = db.everything_14ers_db
data = brewery_mountain_df.to_dict(orient = "records")
everything_14ers.insert_many(data)

,Brewery Name,Brewery Address,Mountain Peak,Brewery Distance From Mountain Peak,Brewery Longitudes,Brewery Latitudes
0,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",Mt. Elbert,12.059106,-106.290761,39.244657
1,Aspen Brewing Company,"304 E Hopkins Ave, Aspen, CO 81611",Mt. Massive,18.455272,-106.820301,39.190323
2,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",Mt. Massive,10.658761,-106.290761,39.244657
3,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",La Plata Peak,17.790090,-106.290761,39.244657
4,Ouray Brewing,"607 Main Street, Ouray, CO 81427",Uncompahgre Peak,11.861862,-107.671124,38.022940


In [51]:
hot_springs = pd.read_csv("../Resources/hot_Springs1.csv", encoding = "utf-8")
hot_springs = hot_springs[["0", "1", "2", "3"]]
hot_springs = hot_springs.rename(columns = ({
    "0" : "Hot Spring Name",
    "1" : "Town", 
    "2" : "Hot Spring Latitude", 
    "3" : "Hot Spring Longitude"}))
display(hot_springs.head())

,Hot Spring Name,Town,Hot Spring Latitude,Hot Spring Longitude
0,Strawberry Park Hot Springs,Steamboat Springs,40.559761,-106.849431
1,The Springs Resort and Spa,Pagosa Springs,37.264588,-107.010919
2,Waunita Hot Springs Ranch,Gunnison,38.515375,-106.508695
3,Glenwood Hot Springs Resort,Glenwood Springs,39.550377,-107.322922
4,Avalanche Ranch Cabins & Hot Springs,Redstone,39.247890,-107.237298


In [52]:
hs_distance_list = []
mountain_hs_list = []
hs_lats = []
hs_longs = []
hs_names = []
hs_towns = []
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k, o in zip(hot_springs["Hot Spring Latitude"], hot_springs["Hot Spring Longitude"], hot_springs["Hot Spring Name"], hot_springs["Town"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_hs_list.append(l)
            hs_distance_list.append(distance)
            hs_lats.append(i)
            hs_longs.append(j)
            hs_names.append(Name)
            hs_towns.append(o)

In [53]:
hs_mountain_df = pd.DataFrame({
    "Hot Spring Name" : hs_names,
    "Town" : hs_towns,
    "Mountain Peak" : mountain_hs_list,
    "HS Distance From Mountain Peak" : hs_distance_list,
    "Hot Spring Longitudes" : hs_longs,
    "Hot Spring Latitudes" : hs_lats,
})
display(hs_mountain_df.head())

everything_14ers = db.everything_14ers_db
data = hs_mountain_df.to_dict(orient = "records")
everything_14ers.insert_many(data)

,Hot Spring Name,Town,Mountain Peak,HS Distance From Mountain Peak,Hot Spring Longitudes,Hot Spring Latitudes
0,Mount Princeton Hot Springs Resort,Nathrop,Mt. Harvard,15.748262,-106.162633,38.732602
1,Aqua Hot Spring & Cabin,Buena Vista,Mt. Harvard,15.948175,-106.173609,38.724037
2,Antero Hot Springs Cabins,Nathrop,Mt. Harvard,15.678607,-106.167117,38.731589
3,Cottonwood Hot Springs Inn & Spa,Buena Vista,Mt. Harvard,9.369500,-106.221819,38.812768
4,Great Sand Dunes Swimming Pool,Alamosa,Blanca Peak,10.844181,-105.512504,37.732996


MergeMania!!

In [19]:
#first merge included distance from mountain peak data
first_dt_mg = pd.merge(cleaned_mountains,charger_mountain_df, on = "Mountain Peak" )
second_dt_mg = pd.merge(first_dt_mg,gas_mountain_df, on = "Mountain Peak")
third_dt_mg = pd.merge(second_dt_mg, brewery_mountain_df, on = "Mountain Peak")
fourth_dt_mg = pd.merge(third_dt_mg, hs_mountain_df, on = "Mountain Peak")
display(fourth_dt_mg.head())

,Mountain Peak,Distance from Campsite (miles),Campsite Name,Campsite Longitude,Campsite Latitude,Dates,#_Sites,Isolation_mi,Peak Longitude,Peak Latitude,...,Brewery Name,Brewery Address,Brewery Distance From Mountain Peak,Brewery Longitudes,Brewery Latitudes,Hot Spring Name,Town,HS Distance From Mountain Peak,Hot Spring Longitudes,Hot Spring Latitudes
0,Mt. Belford,9.272224,Dexter,-106.325,39.092,,24,3.3,-106.3607,38.9607,...,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",19.974929,-106.290761,39.244657,Mount Princeton Hot Springs Resort,Nathrop,19.025994,-106.162633,38.732602
1,Mt. Belford,9.272224,Dexter,-106.325,39.092,,24,3.3,-106.3607,38.9607,...,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",19.974929,-106.290761,39.244657,Aqua Hot Spring & Cabin,Buena Vista,19.203125,-106.173609,38.724037
2,Mt. Belford,9.272224,Dexter,-106.325,39.092,,24,3.3,-106.3607,38.9607,...,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",19.974929,-106.290761,39.244657,Antero Hot Springs Cabins,Nathrop,18.950307,-106.167117,38.731589
3,Mt. Belford,9.272224,Dexter,-106.325,39.092,,24,3.3,-106.3607,38.9607,...,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",19.974929,-106.290761,39.244657,Cottonwood Hot Springs Inn & Spa,Buena Vista,12.659520,-106.221819,38.812768
4,Mt. Belford,9.272224,Dexter,-106.325,39.092,,24,3.3,-106.3607,38.9607,...,Two Mile Brewing Co,"101 Harrison Ave, Leadville, CO 80461",19.974929,-106.290761,39.244657,Mount Princeton Hot Springs Resort,Nathrop,19.025994,-106.162633,38.732602


In [33]:

mountains

,ID,Mountain Peak,Mountain Range,Elevation_ft,fourteener,Prominence_ft,Isolation_mi,Peak Longitude,Peak Latitude,Standard Route,Standard Route Distance (miles),Elevation Gain_ft,Difficulty,Traffic Low,Traffic High,photo
0,1,Mt. Elbert,Sawatch Range,14440,Y,9093,670.00,-106.445400,39.117800,Northeast Ridge,9.50,4700,Class 1,20000,25000,https://www.14ers.com/photos/mtelbert/peakphot...
1,2,Mt. Massive,Sawatch Range,14428,Y,1961,5.06,-106.475700,39.187500,East Slopes,14.50,4500,Class 2,7000,10000,https://www.14ers.com/photos/mtmassive/peakpho...
2,3,Mt. Harvard,Sawatch Range,14421,Y,2360,14.93,-106.320700,38.924400,South Slopes,14.00,4600,Class 2,5000,7000,https://www.14ers.com/photos/harvardgroup/peak...
3,4,Blanca Peak,Sangre de Cristo Range,14351,Y,5326,103.40,-105.485600,37.577500,Northwest Ridge,17.00,6500,Hard Class 2,1000,3000,https://www.14ers.com/photos/blancagroup/peakp...
4,5,La Plata Peak,Sawatch Range,14343,Y,1836,6.28,-106.472900,39.029400,Northwest Ridge,9.25,4500,Class 2,5000,7000,https://www.14ers.com/photos/laplatapeak/peakp...
5,6,Uncompahgre Peak,San Juan Mountains,14321,Y,4277,85.10,-107.462100,38.071700,South Ridge,7.50,3000,Class 2,3000,5000,https://www.14ers.com/photos/uncompahgrepeak/p...
6,7,Crestone Peak,Sangre de Cristo Range,14300,Y,4554,27.40,-105.585500,37.966900,South Face,14.00,5700,Class 3,1000,3000,https://www.14ers.com/photos/crestonegroup/pea...
7,8,Mt. Lincoln,Mosquito Range,14293,Y,3862,22.60,-106.111600,39.351500,West Ridge,6.00,2600,Class 2,15000,20000,https://www.14ers.com/photos/lincolngroup/peak...
8,9,Castle Peak,Elk Mountains,14279,Y,2365,20.90,-106.861400,39.009700,Northeast Ridge,13.50,4600,Hard Class 2,1000,3000,https://www.14ers.com/photos/castlegroup/peakp...
9,10,Grays Peak,Front Range,14278,Y,2770,25.00,-105.817600,39.633900,North Slopes,8.00,3000,Class 1,25000,30000,https://www.14ers.com/photos/graystorreys/peak...


{'Mountain': ['Mt. Elbert',
  'Mt. Massive',
  'Mt. Harvard',
  'Blanca Peak',
  'La Plata Peak',
  'Uncompahgre Peak',
  'Crestone Peak',
  'Mt. Lincoln',
  'Castle Peak',
  'Grays Peak',
  'Mt. Antero',
  'Torreys Peak',
  'Quandary Peak',
  'Mt. Evans',
  'Longs Peak',
  'Mt. Wilson',
  'Mt. Shavano',
  'Mt. Princeton',
  'Mt. Belford',
  'Crestone Needle',
  'Mt. Yale',
  'Mt. Bross',
  'Kit Carson Mountain',
  'Maroon Peak',
  'Tabeguache Peak',
  'Mt. Oxford (Colorado)',
  'Mt. Sneffels',
  'Mt. Democrat',
  'Capitol Peak',
  'Pikes Peak',
  'Snowmass Mountain',
  'Windom Peak',
  'Mt. Eolus',
  'Challenger Point',
  'Mt. Columbia',
  'Missouri Mountain',
  'Humboldt Peak',
  'Mt. Bierstadt',
  'Sunlight Peak',
  'Handies Peak',
  'Culebra Peak',
  'Ellingwood Point',
  'Mt. Lindsey',
  'Little Bear Peak',
  'Mt. Sherman',
  'Redcloud Peak',
  'Pyramid Peak',
  'Wilson Peak',
  'San Luis Peak',
  'Wetterhorn Peak',
  'Mt. of the Holy Cross',
  'Huron Peak',
  'Sunshine Peak',
  '

We need to group everything by the mountians

In [34]:
# Declare the collection for camping & 14ers records
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.FullData

everything_14ers = db.everything_14ers_db
data = fourth_dt_mg.to_dict(orient = "records")
everything_14ers.insert_many(data)
#mountains database, should be all data for home page layer on leaflet
mountains_db = db.mountains_db
data = mountains.to_dict(orient = "records")
mountains_db.insert_many(data)

In [ ]:
#fire the following DataFrames in Mongo at once, no merge, will display everything 
#(breweries_df, gas_df, charger_df, mountains, co_campsites)